In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Introduction
https://www.kaggle.com/tonysun94/pytorch-1-0-1-on-mnist-acc-99-8/notebook


In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

import cv2
import matplotlib.pyplot as plt

from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

## Exploring Data

In [ ]:

train_images = pd.read_pickle('/kaggle/input/modified-mnist/train_max_x')
train_df_y = pd.read_csv('/kaggle/input/modified-mnist/train_max_y.csv')
test_images = pd.read_pickle('/kaggle/input/modified-mnist/test_max_x')

# have a glimpse of data structrure
n_train = len(train_images)
n_pixels = len(train_images[0])
print('Number of training samples: {0}'.format(n_train))
print('Number of training pixels: {0}'.format(n_pixels))

n_test = len(test_images)
n_pixels = len(test_images[0])
print('Number of test samples: {0}'.format(n_test))
print('Number of test pixels: {0}'.format(n_pixels))

## Displaying Some Images

In [ ]:
random_sel = np.random.randint(n_train, size=8)

grid = make_grid(torch.Tensor((train_images[random_sel]/255.).reshape((-1,128,128))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16,2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.axis('off')
print(*list(train_df_y.iloc[random_sel, 0].values), sep=', ')

## Noise Reduction
Refer to 'Data Cleaning' notebook

In [ ]:
def filter_image(img):
    image = np.array(img, dtype=np.uint8)
    image = (255-image)
    kernel = np.ones((1,1),np.uint8)

    median = cv2.medianBlur(image, 1)

    thresh = cv2.threshold(median.copy(), 60, 255, cv2.THRESH_BINARY)[1]
    
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    return opening

In [ ]:
# x_train_imgs = np.full(train_images.shape, 0)
# x_test_imgs = np.full(test_images.shape, 0)
x_train_imgs = train_images
x_test_imgs = test_images

In [ ]:
# for i in range (0, len(train_images)):
#     x_train_imgs[i] = filter_image(train_images[i])
    
# for i in range (0, len(test_images)):
#     x_test_imgs[i] = filter_image(test_images[i])

## Clean the Data
We will remove some noise using a blurring method and then thresholding

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(x_train_imgs, train_df_y.Label.values.astype(int), test_size=0.1) #random state 42 can be added
print(X_train.shape, X_val.shape)
print(y_train.shape,y_val.shape )
print(y_train[0])

# Construct Data Sets


In [ ]:
from torchvision.transforms import Compose, ToTensor, Normalize, Resize

class ModifiedMNIST(torch.utils.data.Dataset):
    """
    This is our custom dataset class which will load images, perform transforms on them, and load their corresponding labels
    """
    
    def __init__(self, imgs=None, labels=None, transform=None):
        self.X = imgs
        self.y = labels
            
#         self.transform = Compose([Resize((224, 224)), ToTensor(), Normalize((self.X.mean()/255,), (self.X.std()/255,))])
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is not None:
            return self.transform(self.X[idx]), self.y[idx]
        else:
            return self.transform(self.X[idx])
            

In [ ]:
batch_size = 64
RandAffine = transforms.RandomAffine(degrees=20, translate=(0.05, 0.05), scale=(0.9, 1.1))

train_transforms = transforms.Compose(
    [transforms.ToPILImage(),
#      transforms.Grayscale(num_output_channels=1),
     Resize((224, 224)),
     RandAffine,
     transforms.ToTensor(),
     transforms.Normalize((x_train_imgs.mean()/255,), (x_train_imgs.std()/255,))])

val_test_transforms = transforms.Compose(
    [transforms.ToPILImage(),
#      transforms.Grayscale(num_output_channels=1),
     Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((x_train_imgs.mean()/255,), (x_train_imgs.std()/255,))])

# Building Modified ResNet
We will follow this tutorial found online
https://zablo.net/blog/post/using-resnet-for-mnist-in-pytorch-tutorial/index.html


In [ ]:
from torchvision.models.resnet import ResNet, BasicBlock

class MnistResNet(ResNet):
    def __init__(self):
        super(MnistResNet, self).__init__(BasicBlock, [2,2,2,2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64,
                                    kernel_size = (7,7),
                                    stride=(2,2),
                                    padding=(3,3), bias = False)

    
# model = MnistResNet()
# print(model)

# Creating an Ensemble

In [ ]:
models = []
models.append(MnistResNet())
models.append(MnistResNet())
models.append(MnistResNet())
models.append(MnistResNet())
# models.append(MnistResNet())
# models.append(MnistResNet())
# models.append(MnistResNet())

# Training and Evaluation

In [ ]:
from torch.autograd import Variable
def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        
        if (batch_idx + 1)%100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.data.item()))

In [ ]:
def validate(val_loader, model, criterion):
    model.eval()
    loss = 0
    correct = 0
    
    for _, (data, target) in enumerate(val_loader):
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        
        loss += criterion(output, target).data.item()

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(val_loader.dataset)
        
    print('\nOn Val set Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(val_loader.dataset),
        100.0 * float(correct) / len(val_loader.dataset)))

In [ ]:
# example config, use the comments to get higher accuracy
total_epoches = 20 # 50
step_size = 5     # 10
base_lr = 0.01    # 0.01

# optimizer = optim.Adam(model.parameters(), lr=base_lr)
# criterion = nn.CrossEntropyLoss()
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

# if torch.cuda.is_available():
#     model = model.cuda()
#     criterion = criterion.cuda()

In [ ]:
for i in range(len(models)):
    train_dataset = ModifiedMNIST(X_train, y_train, transform=train_transforms)
    val_dataset = ModifiedMNIST(X_val, y_val, transform=val_test_transforms)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                            batch_size=batch_size, shuffle=False)
    
    model = models[i]
    optimizer = optim.Adam(model.parameters(), lr=base_lr)
    criterion = nn.CrossEntropyLoss()
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

    if torch.cuda.is_available():
        model = model.cuda()
        criterion = criterion.cuda()
    for epoch in range(total_epoches):
        print("\nTrain Epoch {}: lr = {}".format(epoch, exp_lr_scheduler.get_lr()[0]))

        train(train_loader=train_loader, model=model, criterion=criterion, optimizer=optimizer, epoch=epoch)
        validate(val_loader=val_loader, model=model, criterion=criterion)
        exp_lr_scheduler.step()
    

In [ ]:
# for epoch in range(total_epoches):
#     print("\nTrain Epoch {}: lr = {}".format(epoch, exp_lr_scheduler.get_lr()[0]))
    
#     train_dataset = ModifiedMNIST(X_train, y_train, transform=train_transforms)
#     val_dataset = ModifiedMNIST(X_val, y_val, transform=val_test_transforms)

#     train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                                batch_size=batch_size, shuffle=True)
#     val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
#                                              batch_size=batch_size, shuffle=False)

#     train(train_loader=train_loader, model=model, criterion=criterion, optimizer=optimizer, epoch=epoch)
#     validate(val_loader=val_loader, model=model, criterion=criterion)
#     exp_lr_scheduler.step()

# Making predictions

In [ ]:
def prediction(test_loader, model):
    model.eval()
    test_pred = torch.LongTensor()
    
    for i, data in enumerate(test_loader):
        if torch.cuda.is_available():
            data = data.cuda()
            
        output = model(data)
        
        pred = output.cpu().data.max(1, keepdim=True)[1]
        test_pred = torch.cat((test_pred, pred), dim=0)
        
    return test_pred

In [ ]:
from scipy import stats

test_batch_size = 64
test_dataset = ModifiedMNIST(imgs=test_images, transform=val_test_transforms)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=test_batch_size, shuffle=False)

# tensor prediction
labels = []
for model in models:
    model_pred = prediction(test_loader, model)
    labels.append(model_pred.numpy())

    
labels = np.array(labels)
# labels = np.transpose(labels, (1,0))
labels = stats.mode(labels)[0]
# lables = np.squeeze(labels)
labels = labels[0]
print(labels.shape)

labels = np.transpose(labels, (1,0))
labels = np.squeeze(labels)

test_pred_df = pd.DataFrame({'Id' : np.arange(0, model_pred.shape[0]), 'Label' : labels})

# test_pred = prediction(test_loader, model)

# # tensor -> numpy.ndarray -> pandas.DataFrame
# test_pred_df = pd.DataFrame(np.c_[np.arange(0, len(test_dataset)), test_pred.numpy()], 
#                       columns=['Id', 'Label'])

# # show part of prediction dataframe
print(test_pred_df.head())

In [ ]:
grid = make_grid(torch.Tensor((x_train_imgs[random_sel]/255.).reshape((-1,128,128))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16,2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.axis('off')
print(*list(train_df_y.iloc[[0,1,2,3,4,5,6,7], 0].values), sep=', ')

In [ ]:
print(len(test_pred_df))

In [ ]:
# file.remove("/kaggle/working/submission_RESNET18.csv")

In [ ]:
test_pred_df.to_csv('submission_ensemble.csv', index=False)

# Saving Model for Future Use

In [ ]:
model = models[0]
checkpoint = {'model': MnistResNet(),
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint1.pth')

model = models[1]
checkpoint = {'model': MnistResNet(),
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint2.pth')

model = models[2]
checkpoint = {'model': MnistResNet(),
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint3.pth')

model = models[3]
checkpoint = {'model': MnistResNet(),
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint4.pth')

# model = models[4]
# checkpoint = {'model': MnistResNet(),
#               'state_dict': model.state_dict(),
#               'optimizer' : optimizer.state_dict()}

# torch.save(checkpoint, 'checkpoint5.pth')

# model = models[5]
# checkpoint = {'model': MnistResNet(),
#               'state_dict': model.state_dict(),
#               'optimizer' : optimizer.state_dict()}

# torch.save(checkpoint, 'checkpoint6.pth')

# model = models[6]
# checkpoint = {'model': MnistResNet(),
#               'state_dict': model.state_dict(),
#               'optimizer' : optimizer.state_dict()}

# torch.save(checkpoint, 'checkpoint7.pth')



In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model

In [ ]:
model = load_checkpoint('checkpoint1.pth')
print(model)
model = load_checkpoint('checkpoint2.pth')
print(model)
model = load_checkpoint('checkpoint3.pth')
print(model)